In [14]:
import pandas as pd
import numpy as np
import ta
from google.cloud import storage
#import functions_framework


btc_df = pd.read_csv("historical_bitcoin_data.csv")
btc_df['ds'] = pd.to_datetime(btc_df['ds'] )

In [15]:
btc_df.shape, btc_df['ds'].min(), btc_df['ds'].max()

((48984, 10),
 Timestamp('2019-01-01 00:00:00'),
 Timestamp('2024-08-02 23:00:00'))

In [16]:


btc_df['RSI'] = ta.momentum.RSIIndicator(btc_df['y']).rsi()

macd = ta.trend.MACD(btc_df['y'])
btc_df['MACD'] = macd.macd()
btc_df['MACD_signal'] = macd.macd_signal()
btc_df['MACD_diff'] = macd.macd_diff()


# Calculate typical price for MFI
btc_df['typical_price'] = (btc_df['y_upper'] + btc_df['y_lower'] + btc_df['y']) / 3

# Calculate raw money flow
btc_df['raw_money_flow'] = btc_df['typical_price'] * btc_df['volume']

# Calculate positive and negative money flow
btc_df['positive_money_flow'] = np.where(btc_df['typical_price'] > btc_df['typical_price'].shift(1), btc_df['raw_money_flow'], 0)
btc_df['negative_money_flow'] = np.where(btc_df['typical_price'] < btc_df['typical_price'].shift(1), btc_df['raw_money_flow'], 0)


# Calculate money flow ratio
positive_money_flow_rolling = btc_df['positive_money_flow'].rolling(window=14).sum()
negative_money_flow_rolling = btc_df['negative_money_flow'].rolling(window=14).sum()
money_flow_ratio = positive_money_flow_rolling / negative_money_flow_rolling

# Calculate MFI
btc_df['MFI'] = 100 - (100 / (1 + money_flow_ratio))


# Bollinger Bands for volatility indicator
rolling_mean = btc_df['y'].rolling(window=20).mean()
rolling_std = btc_df['y'].rolling(window=20).std()
btc_df['bollinger_upper'] = rolling_mean + (rolling_std * 2)
btc_df['bollinger_lower'] = rolling_mean - (rolling_std * 2)

In [17]:
btc_df.shape, btc_df['ds'].min(), btc_df['ds'].max()

((48984, 21),
 Timestamp('2019-01-01 00:00:00'),
 Timestamp('2024-08-02 23:00:00'))

In [19]:
# Show records with missing values
btc_df[btc_df.isnull().any(axis=1)]

,ds,open_price,y_upper,y_lower,y,volume,quote_asset_volume,num_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,...,MACD,MACD_signal,MACD_diff,typical_price,raw_money_flow,positive_money_flow,negative_money_flow,MFI,bollinger_upper,bollinger_lower
0,2019-01-01 00:00:00,3701.23,3713.00,3689.88,3700.31,686.367420,2.539069e+06,5534,370.855314,1.371962e+06,...,NaN,NaN,NaN,3701.063333,2.540289e+06,0.000000e+00,0.000000e+00,NaN,NaN,NaN
1,2019-01-01 01:00:00,3700.20,3702.73,3684.22,3689.69,613.539115,2.266700e+06,5086,320.644448,1.184519e+06,...,NaN,NaN,NaN,3692.213333,2.265317e+06,0.000000e+00,2.265317e+06,NaN,NaN,NaN
2,2019-01-01 02:00:00,3689.67,3695.95,3675.04,3690.00,895.302181,3.302044e+06,6391,471.857118,1.740469e+06,...,NaN,NaN,NaN,3686.996667,3.300976e+06,0.000000e+00,3.300976e+06,NaN,NaN,NaN
3,2019-01-01 03:00:00,3690.00,3699.77,3685.78,3693.13,796.714818,2.942422e+06,5709,459.948381,1.698857e+06,...,NaN,NaN,NaN,3692.893333,2.942183e+06,2.942183e+06,0.000000e+00,NaN,NaN,NaN
4,2019-01-01 04:00:00,3692.32,3720.00,3685.94,3692.71,1317.452909,4.872937e+06,7908,770.995533,2.852106e+06,...,NaN,NaN,NaN,3699.550000,4.873983e+06,4.873983e+06,0.000000e+00,NaN,NaN,NaN
5,2019-01-01 05:00:00,3693.98,3704.44,3692.73,3699.94,572.402228,2.117705e+06,3886,326.520975,1.208150e+06,...,NaN,NaN,NaN,3699.036667,2.117337e+06,0.000000e+00,2.117337e+06,NaN,NaN,NaN
6,2019-01-01 06:00:00,3699.95,3713.93,3697.00,3703.56,660.279771,2.446401e+06,4839,384.509915,1.424667e+06,...,NaN,NaN,NaN,3704.830000,2.446224e+06,2.446224e+06,0.000000e+00,NaN,NaN,NaN
7,2019-01-01 07:00:00,3703.63,3726.64,3703.34,3713.83,823.625491,3.057514e+06,5335,424.739220,1.577037e+06,...,NaN,NaN,NaN,3714.603333,3.059442e+06,3.059442e+06,0.000000e+00,NaN,NaN,NaN
8,2019-01-01 08:00:00,3714.19,3731.19,3707.00,3716.70,887.101362,3.300500e+06,6190,468.473211,1.742964e+06,...,NaN,NaN,NaN,3718.296667,3.298506e+06,3.298506e+06,0.000000e+00,NaN,NaN,NaN
9,2019-01-01 09:00:00,3716.98,3732.00,3696.14,3699.95,955.879034,3.545474e+06,5670,510.885391,1.895818e+06,...,NaN,NaN,NaN,3709.363333,3.545703e+06,0.000000e+00,3.545703e+06,NaN,NaN,NaN


In [20]:

btc_df.dropna(inplace=True)
btc_df.shape, btc_df['ds'].min(), btc_df['ds'].max()

((48951, 21),
 Timestamp('2019-01-02 09:00:00'),
 Timestamp('2024-08-02 23:00:00'))

# Bitcoin_Data

In [21]:
btc_new = pd.read_csv("bitcoin_data.csv")
btc_new['ds'] = pd.to_datetime(btc_new['ds'] )
btc_new.shape, btc_new['ds'].min(), btc_new['ds'].max()

((48951, 21),
 Timestamp('2019-01-02 09:00:00'),
 Timestamp('2024-08-02 23:00:00'))